In [26]:
import pandas as pd
import string
import re

In [12]:
!twarc2 flatten '/home/fabrizio/results.jsonl' > '/home/fabrizio/results_flat.jsonl'

In [13]:
!twarc2 csv '/home/fabrizio/results_flat.jsonl' > '/home/fabrizio/tweets_musk.csv'

In [33]:
data = pd.read_csv('/home/fabrizio/tweets_musk.csv')

/tmp/ipykernel_73796/3157322269.py:1: DtypeWarning: Columns (24,25,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/fabrizio/tweets_musk.csv')


In [34]:
data[data['lang'] == 'en'][['text']]

,text
0,@elonmusk Nobody will ever convince me he's no...
3,Evidence of one of the biggest political scand...
4,@littledude88888 @elonmusk Christ? a lame bast...
5,@elonmusk Merry Squidmas from the #SquidGrowAr...
6,@elonmusk A pair of rats.
...,...
200002,One of the best things @elonmusk did for Canad...
200004,Ways Twitter has changed since @elonmusk took ...
200005,@elonmusk Indeed https://t.co/ErDODa5rJp
200006,This is getting boring… release the whole thin...


In [35]:
data = data[data['lang'] == 'en'][['text']]

In [36]:
data.shape

(141205, 1)

In [37]:
data.to_csv('/home/fabrizio/tweets_musk_text.csv', index=False)

In [38]:
data = pd.read_csv('/home/fabrizio/tweets_musk_text.csv')
data

,text
0,@elonmusk Nobody will ever convince me he's no...
1,Evidence of one of the biggest political scand...
2,@littledude88888 @elonmusk Christ? a lame bast...
3,@elonmusk Merry Squidmas from the #SquidGrowAr...
4,@elonmusk A pair of rats.
...,...
141200,One of the best things @elonmusk did for Canad...
141201,Ways Twitter has changed since @elonmusk took ...
141202,@elonmusk Indeed https://t.co/ErDODa5rJp
141203,This is getting boring… release the whole thin...


In [45]:
def preprocess(path: str):

  # Load data from .csv file
  data = pd.read_csv(path)

  # Drop NA and empty strings
  data = data.dropna(inplace=False)

  # Filter only ASCII characters
  data['text'] = data['text'].apply(lambda x: x.encode("ascii", "ignore").decode())

  # Lower casing text
  data['text'] = data['text'].apply(str.lower)

  # Remove URLs from text
  data['text'] = data['text'].apply(lambda x: re.sub(
      r"http\S+|www\S+|https\S+|([a-z0-9-]+\.)+com|([a-z0-9-]+\.)+org|([a-z0-9-]+\.)+net", "", x, flags=re.MULTILINE))
  
  # Remove user mentions @ and hashtags # from text
  data['text'] = data['text'].apply(lambda x: re.sub(
      r"\@\w+|\#", "", x))
  
  # Remove punctuation
  data['text'] = data['text'].apply(lambda x: x.translate(
      str.maketrans('', '', string.punctuation)))

  # Remove blank text
  blanks = []  
  for index, text in data.itertuples():
      if type(text)==str:            
          if text.isspace():     
              blanks.append(index)     
  data.drop(blanks, inplace=True)

  # Remove multiple, leading and trailing spaces
  data['text'] = data['text'].apply(lambda x: re.sub(" +", " ", x))
  data['text'] = data['text'].str.strip()

  # Add column containing length of text
  data['text_len'] = data['text'].apply(lambda x: sum(len(y) for y in x.split()))

  return data

In [50]:
path = '/home/fabrizio/tweets_musk_text.csv'

data = preprocess(path)

In [51]:
data

,text,text_len
0,nobody will ever convince me hes not one of th...,66
1,evidence of one of the biggest political scand...,110
2,christ a lame bastard that was crucified long ...,177
3,merry squidmas from the squidgrowarmy squidgro...,118
4,a pair of rats,11
...,...,...
141200,one of the best things did for canadians was t...,200
141201,ways twitter has changed since took overnn les...,154
141202,indeed,6
141203,this is getting boring release the whole thing...,46


In [52]:
data.to_csv('/home/fabrizio/tw_musk_clean_text.csv', index=False)